In [ ]:
import pandas as pd
from preProccesPuzzle import PreProcess
from clue_classifier import ClueClassifier
from constraints import Constraint, IdentityConstrain, NextToConstrain, DistanceConstrain,RightConstrain, LeftConstrain, DirectRightConstrain, DirectLeftConstrain, PositionAbsoluteConstrain, PositionAbsoluteNegativeConstrain

In [2]:
gridmode = pd.read_parquet("Gridmode-00000-of-00001.parquet")
mc = pd.read_parquet("mc-00000-of-00001.parquet")
ppp = PreProcess()
classifier = ClueClassifier()

In [ ]:
puzzle = mc.puzzle.iloc[1].lower()

attrs, clues = ppp.proccess(puzzle)

#print(puzzle)

print(attrs.keys())
for k in attrs.keys():
    print(attrs[k])
print()

constrains: list[Constraint] = []

for c in clues:
    clue, clue_type = classifier.classify(c)

    if clue_type == "IDENTITY":
        constrains.append(IdentityConstrain(attrs, clue))
    if clue_type == "NEXT_TO":
        constrains.append(NextToConstrain(attrs, clue))
    if clue_type == "LEFT":
        constrains.append(LeftConstrain(attrs, clue))
    if clue_type == "RIGHT":
        constrains.append(RightConstrain(attrs, clue))
    if clue_type == "DISTANCE":
        constrains.append(DistanceConstrain(attrs, clue))
    if clue_type == "DIRECT_LEFT":
        constrains.append(DirectLeftConstrain(attrs, clue))
    if clue_type == "DIRECT_RIGHT":
        constrains.append(DirectRightConstrain(attrs, clue))
    if clue_type == "POSITION_ABSOLUTE":
        constrains.append(PositionAbsoluteConstrain(attrs, clue))
    if clue_type == "POSITION_ABSOLUTE_NEGATIVE":
        constrains.append(PositionAbsoluteNegativeConstrain(attrs, clue))
    if clue_type == "UNKNOWN":
        raise TypeError




dict_keys(['name', 'education', 'vacation', 'music', 'occupation', 'sports', 'house nr.'])
['eric', 'peter', 'arnold']
['associate', 'high school', 'bachelor']
['city', 'beach', 'mountain']
['classical', 'rock', 'pop']
['engineer', 'doctor', 'teacher']
['tennis', 'basketball', 'soccer']
['1', '2', '3']

